# Spacecraft trajectories

This notebook shows how to calculate the trajectory of the JUICE and INTEGRAL spacecraft around the Sun, Earth, Mars and Jupiter. 

It uses the `planetary_coverage` package to build the tour configurations and calculate the trajectories. You can find more information about the project in the [documentation](https://planetary-coverage.readthedocs.io/en/latest/).

## Prerequisites

### Install the required packages (optional)

Before running the code cells below, make sure to install the required packages.

In [ ]:
!pip install planetary-coverage
!pip install pandas
!pip install matplotlib
!pip install tables

### Import the required packages

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from planetary_coverage import TourConfig
from datetime import datetime, timedelta

### Helper functions

In [7]:
def km_to_au(km: float) -> float:
    """Convert kilometers to astronomical units.
    """
    return km * 6.68459e-9

## JUICE trajectory calculations

In this section we will calculate the trajectory of the JUICE spacecraft around the Sun, Earth, Mars and Jupiter and plot the results.

> ℹ️ The first execution of *Calculating* code cell will download the SPICE kernels to `RADEM_KERNELS_DIR` directory if they are not present. This may take a while and is only done once.

In [8]:
JUICE_KERNELS_DIR = '../data/spice' # Directory to download kernels to
JUICE_KERNELS_MK = '5.1 150lb_23_1' # Kernel set to use
JUICE_KERNELS_VERSION = 'v451' # Kernel version

START_DATE = datetime(2023, 9, 1) # Start date of the trajectory
END_DATE = datetime(2033, 9, 1) # End date of the trajectory
INTERVAL_TIME = timedelta(hours=12) # Interval between trajectory points

### Helper functions

In [9]:
def build_juice_tour(target):
    """Build a JUICE tour configuration for a specific target.
    """
    return TourConfig(
        spacecraft='JUICE',
        instrument='RADEM_PSD',
        target=target,
        download_kernels=True,
        kernels_dir=JUICE_KERNELS_DIR,
        mk=JUICE_KERNELS_MK,
        version=JUICE_KERNELS_VERSION,
    )
    
def build_trajectory(tour: TourConfig, 
                     start_date: datetime = START_DATE, 
                     end_date: datetime = END_DATE, 
                     interval_time: timedelta = INTERVAL_TIME):
    """Build a trajectory for a given tour configuration.
    """
    interval = f'{interval_time.total_seconds() / 3600} hours'
    return tour[start_date: end_date: interval]

### Calculations

First we build the tour configurations for each target and calculate the trajectories.

In [10]:
sun_tour = build_juice_tour('SUN')
earth_tour = build_juice_tour('EARTH')
mars_tour = build_juice_tour('MARS')
jupiter_tour = build_juice_tour('JUPITER')

sun_traj = build_trajectory(sun_tour)
earth_traj = build_trajectory(earth_tour)
mars_traj = build_trajectory(mars_tour)
jupiter_traj = build_trajectory(jupiter_tour)

Now, we calculate the distances, right ascensions and declinations for each trajectory point.

In [11]:
sun_dist = km_to_au(sun_traj.dist)
earth_dist = km_to_au(earth_traj.dist)
mars_dist = km_to_au(mars_traj.dist)
jupiter_dist = km_to_au(jupiter_traj.dist)

sun_ra = sun_traj.ra
earth_ra = earth_traj.ra
mars_ra = mars_traj.ra
jupiter_ra = jupiter_traj.ra

sun_dec = sun_traj.dec
earth_dec = earth_traj.dec
mars_dec = mars_traj.dec
jupiter_dec = jupiter_traj.dec

It would be easier to work with the data if we put it in a pandas DataFrame.

In [ ]:
df_juice = pd.DataFrame({
    'time': sun_traj.utc,
    'sun_dist_au': sun_dist,
    'earth_dist_au': earth_dist,
    'mars_dist_au': mars_dist,
    'jupiter_dist_au': jupiter_dist,
    'sun_ra': sun_ra,
    'earth_ra': earth_ra,
    'mars_ra': mars_ra,
    'jupiter_ra': jupiter_ra,
    'sun_dec': sun_dec,
    'earth_dec': earth_dec,
    'mars_dec': mars_dec,
    'jupiter_dec': jupiter_dec,
})

df_juice.head()

### Plotting the results

Let's plot the distances to all targets as a function of time.

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(df_juice['time'], df_juice['sun_dist_au'], label='Sun')
plt.plot(df_juice['time'], df_juice['earth_dist_au'], label='Earth')
plt.plot(df_juice['time'], df_juice['mars_dist_au'], label='Mars')
plt.plot(df_juice['time'], df_juice['jupiter_dist_au'], label='Jupiter')

plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Distance (AU)')
plt.title('Distance to Solar System Bodies')
plt.legend()

# Rotate and align the tick labels so they look better
plt.gcf().autofmt_xdate()

# Use AutoDateLocator for smart date tick selection
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.tight_layout()
plt.show()


Let's plot the right ascensions and declinations for the Sun.


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot Right Ascension
ax1.plot(df_juice['time'], df_juice['sun_ra'], label='Sun')
ax1.grid(True)
ax1.set_ylabel('Right Ascension (degrees)')
ax1.set_title('Right Ascension of Sun')
ax1.legend()
ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Plot Declination
ax2.plot(df_juice['time'], df_juice['sun_dec'], label='Sun')
ax2.grid(True)
ax2.set_xlabel('Time')
ax2.set_ylabel('Declination (degrees)')
ax2.set_title('Declination of Sun')
ax2.legend()
ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Rotate date labels
fig.autofmt_xdate()

plt.tight_layout()
plt.show()

## INTEGRAL trajectory calculations

In this section we will calculate the trajectory of the Integral spacecraft around the Sun and Earth.

> ℹ️ The first execution of *Calculating* code cell will download the SPICE kernels to `INTEGRAL_KERNELS_DIR` directory if they are not present. This may take a while and is only done once.

> ⚠️ There is an issue with the official SPICE kernels for INTEGRAL. Some of the data is missing which makes the trajectory calculations impossible for some time ranges As of 2024-12-10, calculating distances from the INTEGRAL is available for the following time ranges:
> - from 2002 OCT 17 05:45:07.900 to 2022 OCT 16 17:16:57.759
> - from 2022 OCT 19 09:07:11.840 to 2022 NOV 14 23:36:51.214
> - from 2024 SEP 26 00:21:02.190 to 2024 OCT 06 15:42:55.360
> 
> It relies on the `spk/integral_sc_ssm_20021017_20241006_v01.bsp` SPICE kernel.  The problem requires further investigation.

In [15]:
INTEGRAL_KERNELS_DIR = '../data/irem/spice' # Directory to download kernels to
INTEGRAL_KERNELS_MK = 'ops' # Kernel set to use
INTEGRAL_KERNELS_VERSION = 'v001' # Kernel version

START_DATE = datetime(2013, 9, 1) # Start date of the trajectory
END_DATE = datetime(2014, 9, 1) # End date of the trajectory
INTERVAL_TIME = timedelta(hours=12) # Interval between trajectory points

### Helper functions

In [16]:
def build_irem_tour(target):
    """Build a INTEGRAL tour configuration for a specific target.
    """
    return TourConfig(
        spacecraft='INTEGRAL',
        instrument='',
        target=target,
        download_kernels=True,
        kernels_dir=INTEGRAL_KERNELS_DIR,
        mk=INTEGRAL_KERNELS_MK,
        version=INTEGRAL_KERNELS_VERSION,
    )
    
def build_irem_trajectory(tour: TourConfig, 
                          start_date: datetime = START_DATE, 
                          end_date: datetime = END_DATE, 
                          interval_time: timedelta = INTERVAL_TIME):
    """Build a trajectory for a given tour configuration.
    """
    interval = f'{interval_time.total_seconds() / 3600} hours'
    return tour[start_date: end_date: interval]

### Calculations

In [17]:
sun_tour = build_irem_tour('SUN')
earth_tour = build_irem_tour('EARTH')

sun_traj = build_irem_trajectory(sun_tour)
earth_traj = build_irem_trajectory(earth_tour)

sun_dist = km_to_au(sun_traj.dist)
earth_dist = km_to_au(earth_traj.dist)
earth_dist_km = earth_traj.dist

In [ ]:
df_integral = pd.DataFrame({
    'time': sun_traj.utc,
    'sun_dist_au': sun_dist,
    'earth_dist_au': earth_dist,
    'earth_dist_km': earth_dist_km,
})

df_integral.head()

### Plotting the results

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_integral['time'], df_integral['sun_dist_au'], label='Sun Distance')
plt.xlabel('Time')
plt.ylabel('Distance (AU)')
plt.title('Distance from Sun')
plt.legend()
plt.grid(True)

plt.figure(figsize=(12, 6))
plt.plot(df_integral['time'], df_integral['earth_dist_km'], label='Earth Distance')
plt.xlabel('Time')
plt.ylabel('Distance (km)')
plt.title('Distance from Earth')
plt.legend()
plt.grid(True)

## Saving to HDF5

Save the processed results to HDF5 files for further analysis.

In [20]:
df_juice.to_hdf('../data/juice_trajectory.h5', 
                key='trajectory', 
                mode='w',
                format='table')

In [21]:
df_integral.to_hdf('../data/integral_trajectory.h5', 
                   key='trajectory', 
                   mode='w',
                   format='table')

## Saving to CSV

Save the processed results to CSV files for further analysis.

In [22]:
df_juice.to_csv('../data/juice_trajectory.csv', index=False)

In [23]:
df_integral.to_csv('../data/integral_trajectory.csv', index=False)

## Reading from HDF5

In [ ]:
df_juice = pd.read_hdf('../data/juice_trajectory.h5', key='trajectory')
df_integral = pd.read_hdf('../data/integral_trajectory.h5', key='trajectory')

df_integral.head()

## Reading from CSV

In [ ]:
df_juice = pd.read_csv('../data/juice_trajectory.csv')
df_integral = pd.read_csv('../data/integral_trajectory.csv')

df_integral.head()